In [1]:
%display latex

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os

In [3]:
eps = var('eps', latex_name = '\\varepsilon')
t = var('t')
r = var('r')
th = var('th', latex_name = '\\theta')
ph = var('ph', latex_name = '\\varphi')

coords = (t,r,th,ph)

h = function('h')(r,th)
k = function('k')(r,th)
m = function('m')(r,th)
omega = function('omega', latex_name='\\omega')(r,th)

nu = function('nu', latex_name='\\nu')(r)
lamb = function('lamb', latex_name='\\lambda')(r)

Omegae = function('Omegaevalue')(eps)
Ne = function('Ne', latex_name='N_e')(r,eps)

In [4]:
def cosmotensors(gedd):
    
    geuu = 1/gedd
    
    chris = [[[0 for k in range(0,4)] for j in range(0,4)] for i in range(0,4)]
    for i in range(0,4):
        for j in range(0,4):
            for k in range(0,4):
                chris[i][j][k] = 0.5*sum(geuu[i,dd]*(derivative(gdd[dd,j],coords[k])
                                                  + derivative(gdd[dd,k],coords[j])
                                                  - derivative(gdd[j,k],coords[dd])) for dd in range(0,4))
                
    riem = [[[[0 for l in range(0,4)] for k in range(0,4)] for j in range(0,4)] for i in range(0,4)]
    for i in range(0,4):
        for j in range(0,4):
            for k in range(0,4):
                for l in range(0,4):
                    riem[i][j][k][l] = (derivative(chris[i][j][l],coords[k])
                                      - derivative(chris[i][j][k],coords[l])
                                      + sum(chris[i][k][dd]*chris[dd][l][j]
                                          - chris[i][l][dd]*chris[dd][k][j] for dd in range(0,4)))
                    
    ricci = matrix(SR, 4, 4)
    for i in range(0,4):
        for k in range(0,4):
            ricci[i,k] = sum(riem[dd][i][dd][k] for dd in range(0,4))
            
    s_curv = sum(sum(ricci[i,j]*geuu[i,j] for i in range(0,4)) for j in range(0,4))
    
    eins = matrix(SR, 4, 4)
    eins = ricci - 0.5*gedd*s_curv
    
    return {'inverse_metric': simplify(geuu),
            'christoffel': simplify(chris),
            'riemann': simplify(riem),
            'ricci': simplify(ricci),
            'curvature': simplify(s_curv),
            'einstein': simplify(eins)}

In [5]:
start_time = time.time()

gdd = matrix(SR, 4, 4)

gdd[0,0] = -exp(nu)
gdd[1,1] = exp(lamb)
gdd[2,2] = r^2
gdd[3,3] = r^2*sin(th)^2

K1 = matrix(SR, 4, 4)
K1[0,3] = omega*r^2*sin(th)^2
K1[3,0] = omega*r^2*sin(th)^2

K2 = matrix(SR, 4, 4)
K2[0,0] = -4*exp(nu)*h + 2*r^2*sin(th)^2*omega^2
K2[1,1] = 4*exp(lamb)*m
K2[2,2] = 4*k*r^2
K2[3,3] = 4*k*r^2*sin(th)^2

gedd = matrix(SR, 4, 4)
for i in range(0,4):
    for j in range(0,4):
        gedd[i,j] = gdd[i,j] + eps*K1[i,j] + 1/2*eps^2*K2[i,j]

tensors = cosmotensors(gedd)
geuu = tensors['inverse_metric']
ricci = tensors['ricci']
s_curv = tensors['curvature']
eins = tensors['einstein']                                                                               #G_\mu\nu e

deins = matrix(SR, 4, 4)
ddeins = matrix(SR, 4, 4)
for i in range(0,4):
    for j in range(0,4):
        deins[i,j] = diff(eins[i,j],eps)
        ddeins[i,j] = diff(eins[i,j],eps,eps)
        
print("--- %s seconds ---" % (time.time() - start_time))

--- 8.09479284286499 seconds ---


In [6]:
print("ge0 = ")
print(gdd)
print("K1 = ")
print(K1)
print("K2 = ")
print(K2)
print("ge = ")
print(gedd)

ge0 = 
[     -e^nu(r)             0             0             0]
[            0     e^lamb(r)             0             0]
[            0             0           r^2             0]
[            0             0             0 r^2*sin(th)^2]
K1 = 
[                         0                          0                          0 r^2*omega(r, th)*sin(th)^2]
[                         0                          0                          0                          0]
[                         0                          0                          0                          0]
[r^2*omega(r, th)*sin(th)^2                          0                          0                          0]
K2 = 
[2*r^2*omega(r, th)^2*sin(th)^2 - 4*e^nu(r)*h(r, th)                                                   0                                                   0                                                   0]
[                                                  0                                4*e^lamb(r)*m(r

In [7]:
# Para imprimir "más bonito":

show("ge0 = ", gdd[:])
show("K1 = ", K1[:])
show("K2 = ", K2[:])
show("ge = ", gedd[:])

'ge0 = ' [     -e^nu(r)             0             0             0]
[            0     e^lamb(r)             0             0]
[            0             0           r^2             0]
[            0             0             0 r^2*sin(th)^2]

'K1 = ' [                         0                          0                          0 r^2*omega(r, th)*sin(th)^2]
[                         0                          0                          0                          0]
[                         0                          0                          0                          0]
[r^2*omega(r, th)*sin(th)^2                          0                          0                          0]

'K2 = ' [2*r^2*omega(r, th)^2*sin(th)^2 - 4*e^nu(r)*h(r, th)                                                   0                                                   0                                                   0]
[                                                  0                                4*e^lamb(r)*m(r, th)                                                   0                                                   0]
[                                                  0                                                   0                                      4*r^2*k(r, th)                                                   0]
[                                                  0                                                   0                                                   0                            4*r^2*k(r, th)*sin(th)^2]

'ge = ' [(r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*eps^2 - e^nu(r)                                                                   0                                                                   0                                      eps*r^2*omega(r, th)*sin(th)^2]
[                                                                  0                              2*eps^2*e^lamb(r)*m(r, th) + e^lamb(r)                                                                   0                                                                   0]
[                                                                  0                                                                   0                                          2*eps^2*r^2*k(r, th) + r^2                                                                   0]
[                                     eps*r^2*omega(r, th)*sin(th)^2                                                                   0                                                                   0                      2*eps^2*r^2*k(r, th)*sin(th)^2 + r^2*sin(th)^2]

In [8]:
#4-velocities of the fluid for any epsilon

ue = [Ne, 0, 0, Ne*Omegae]
u = [exp(-nu/2), 0, 0, 0]

Nevalue = solve(u[0]==ue[0], Ne)[0].rhs()                                                    #Eq 4.20
Omegaevalue = solve(u[3]==ue[3], Omegae)[0].rhs()                                            #Eq 4.20

In [9]:
show(LatexExpr("N_{\\varepsilon}\lvert_0 = "), Nevalue)
show(LatexExpr("\\Omega_{\\varepsilon}\lvert_0 = "), Omegaevalue)

N_{\varepsilon}\lvert_0 =  e^(-1/2*nu(r))

\Omega_{\varepsilon}\lvert_0 =  0

In [10]:
u1 = [0,0,0,0]
u2 = [0,0,0,0]

u10 = function('u10')(r)
u20 = function('u20')(r)

Omega1 = function('Omega1')(eps)
Omega2 = function('Omega2')(eps)

s0 = eps == 0
s1 = Ne.subs(eps==0) == Nevalue
s2 = Omegae.subs(eps==0) == Omegaevalue
s3 = diff(Ne,eps).subs(eps==0) == u10
s4 = diff(Omegae,eps).subs(eps==0) == Omega1
s5 = diff(Ne,eps,eps).subs(eps==0) == u20
s6 = diff(Omegae,eps,eps).subs(eps==0) == Omega2

sustituciones = (s0,s1,s2,s3,s4,s5,s6)

for i in range(0,4):
    u1[i] = diff(ue[i],eps).subs(sustituciones)
    u2[i] = diff(ue[i],eps,eps).subs(sustituciones)

Ue = [0, 0, 0, 0]
for i in range(0,4):
    for j in range(0,4):
        Ue[i] += ue[j]*gedd[j,i]
        
geUeue = 0
for i in range(0,4):
    geUeue += Ue[i]*ue[i]

dgeUeue = diff(geUeue, eps).subs(sustituciones) == 0
u10value = solve(dgeUeue, u10)[0].rhs()
s7 = u10 == u10value
for i in range(0,4):
    u1[i] = u1[i].subs(s7)                                                                                 #Eq 4.21

ddgeUeue = diff(geUeue, eps, eps).subs(sustituciones) == 0
u20value = solve(ddgeUeue, u20)[0].rhs()
s8 = u20 == u20value
for i in range(0,4):
    u2[i] = u2[i].subs(u20==u20value).subs(u10==u10value)                                                  #Eq 4.22

In [11]:
show(LatexExpr("u^{(1)} = "), u1)
show(LatexExpr("u^{(2)} = "), u2)

u^{(1)} =  [0, 0, 0, Omega1(eps)*e^(-1/2*nu(r))]

u^{(2)} =  [((r^2*Omega1(eps)^2 + 2*r^2*Omega1(eps)*omega(r, th) + r^2*omega(r, th)^2)*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-3/2*nu(r)),
 0,
 0,
 Omega2(eps)*e^(-1/2*nu(r))]

In [12]:
for i in range(0,4):
    for j in range(0,4):
        deins[i,j] = deins[i,j].subs(sustituciones)                                             #d/de(G_\mu\nu)|e=0
        ddeins[i,j] = ddeins[i,j].subs(sustituciones)                                       #d^2/de^2(G_\mu\nu)|e=0

In [13]:
Ee = function('Ee', latex_name='E_{\\varepsilon}')(r,eps)
Pe = function('Pe', latex_name='P_{\\varepsilon}')(r,eps)

eins0 = matrix(SR, 4, 4)
Tedd = matrix(SR, 4, 4)
Tdd = matrix(SR, 4, 4)
for i in range(0,4):
    for j in range(0,4):
        eins0[i,j] = eins[i,j].subs(sustituciones)                                                         #G_\mu\nu
        Tedd[i,j] = (Ee + Pe)*Ue[i]*Ue[j] + Pe*gedd[i,j]                                                 #T_\mu\nu e
        Tdd[i,j] = Tedd[i,j].subs(sustituciones)                                                           #T_\mu\nu

In [14]:
show(LatexExpr("T_{\\varepsilon}\lvert_0 ="), Tdd)

T_{\varepsilon}\lvert_0 = [(Ee(r, 0) + Pe(r, 0))*e^nu(r) - Pe(r, 0)*e^nu(r)                                                0                                                0                                                0]
[                                               0                               Pe(r, 0)*e^lamb(r)                                                0                                                0]
[                                               0                                                0                                     r^2*Pe(r, 0)                                                0]
[                                               0                                                0                                                0                           r^2*Pe(r, 0)*sin(th)^2]

In [15]:
kappa = 8*pi
E = function('E')(r,th)
P = function('P')(r,th)

s9 = Ee.subs(eps==0) == E
s10 = Pe.subs(eps==0) == P
sustituciones = (s1,s2,s3,s4,s5,s6,s7,s8,s9,s10)
for i in range(0,4):
    for j in range(0,4):
        Tdd[i,j] = Tdd[i,j].subs(sustituciones)
E_tt = solve(eins0[0,0]==kappa*Tdd[0,0],E)[0].rhs()                                            #Eq 3.7
P_rr = solve(eins0[1,1]==kappa*Tdd[1,1],P)[0].rhs()                                            #Eq 3.8
P_thth = solve(eins0[2,2]==kappa*Tdd[2,2],P)[0].rhs()
P_phph = solve(eins0[3,3]==kappa*Tdd[3,3],P)[0].rhs()
ddnu = solve(P_rr == P_thth, diff(nu,r,r))[0].rhs()                                                          #Eq 3.9

In [16]:
show(LatexExpr("E = "), E_tt)
show(LatexExpr("P_{rr} = "), P_rr)
show(LatexExpr("\\nu'' ="), ddnu)

E =  1/8*(r*e^(-lamb(r))*diff(lamb(r), r) - e^(-lamb(r)) + 1)/(pi*r^2)

P_{rr} =  1/8*(r*e^(-lamb(r))*diff(nu(r), r) + e^(-lamb(r)) - 1)/(pi*r^2)

\nu'' = -1/2*(r^2*diff(nu(r), r)^2 - 2*r*diff(lamb(r), r) - (r^2*diff(lamb(r), r) + 2*r)*diff(nu(r), r) + 4*e^lamb(r) - 4)/r^2

In [17]:
dlamb = solve(E==E_tt,diff(lamb,r))[0].rhs()                                                                 #Eq 3.10
dnu = solve(P==P_rr,diff(nu,r))[0].rhs()                                                                     #Eq 3.11

In [18]:
show(LatexExpr("\\lambda' = "), dlamb)
show(LatexExpr("\\nu' = "), dnu)

\lambda' =  (8*pi*r^2*E(r, th)*e^lamb(r) - e^lamb(r) + 1)/r

\nu' =  (8*pi*r^2*P(r, th)*e^lamb(r) + e^lamb(r) - 1)/r

In [19]:
h0 = function('h0', latex_name='h_0')(r,th)
k0 = function('k0', latex_name='k_0')(r,th)
m0 = function('m0', latex_name='m_0')(r,th)

h2 = function('h2', latex_name='h_2')(r,th)
k2 = function('k2', latex_name='k_2')(r,th)
m2 = function('m2', latex_name='m_2')(r,th)

m = m0 + m2*(3*cos(th)^2-1)/2
k = k0 + k2*(3*cos(th)^2-1)/2
h = h0 + h2*(3*cos(th)^2-1)/2

In [20]:
E1 = function('E1', latex_name='E_1')(r)
E2 = function('E2', latex_name='E_2')(r)
P1 = function('P1', latex_name='P_1')(r)
P2 = function('P2', latex_name='P_2')(r)

s11 = diff(Ee,eps).subs(eps==0) == E1
s12 = diff(Ee,eps,eps).subs(eps==0) == E2
s13 = diff(Pe,eps).subs(eps==0) == P1
s14 = diff(Pe,eps,eps).subs(eps==0) == P2

s15 = m == m0 + m2*(3*cos(th)^2-1)/2
s16 = k == k0 + k2*(3*cos(th)^2-1)/2
s17 = h == h0 + h2*(3*cos(th)^2-1)/2

sustituciones = (s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14)

In [21]:
dTdd = matrix(SR, 4, 4)
ddTdd = matrix(SR, 4, 4)
for i in range(0,4):
    for j in range(0,4):
        dTdd[i,j] = diff(Tedd[i,j],eps)
        ddTdd[i,j] = diff(Tedd[i,j],eps,eps)
    
for i in range(0,4):
    for j in range(0,4):
        dTdd[i,j] = dTdd[i,j].subs(sustituciones)                                                 #d/de(T_\mu\nu)|e=0
        ddTdd[i,j] = ddTdd[i,j].subs(sustituciones)                                           #d^2/de^2(T_\mu\nu)|e=0

In [22]:
dTdd[0,0]

2*(E(r, th) + P(r, th))*e^(3/2*nu(r))*u10(r) + (E1(r) + P1(r))*e^nu(r) - P1(r)*e^nu(r)

In [23]:
# Calculamos ahora la primera derivada de la ecuación

show(deins[1,1]," = ", kappa*dTdd[1,1])
show(deins[2,2]," = ", kappa*dTdd[2,2])
show(deins[3,3]," = ", kappa*dTdd[3,3])

P1value = solve(deins[1,1] == kappa*dTdd[1,1],P1)[0].rhs()

show(LatexExpr("\\text{De aquí sacamos }P_1(r) \\rightarrow P_1(r) = "), P1value)

0 ' = ' 8*pi*P1(r)*e^lamb(r)

0 ' = ' 8*pi*r^2*P1(r)

0 ' = ' 8*pi*r^2*P1(r)*sin(th)^2

\text{De aquí sacamos }P_1(r) \rightarrow P_1(r) =  0

In [38]:
show(deins[0,0]," = ", kappa*dTdd[0,0])

E1value = solve(deins[0,0].subs(sustituciones) == kappa*dTdd[0,0].subs(sustituciones),E1)[0].rhs()
show(LatexExpr("\\text{De aquí sacamos }E_1(r,\\theta) \\rightarrow E_1(r,\\theta) = "), E1value)

0 ' = ' 8*pi*(2*(E(r, th) + P(r, th))*e^(3/2*nu(r))*u10(r) + (E1(r) + P1(r))*e^nu(r) - P1(r)*e^nu(r))

\text{De aquí sacamos }E_1(r,\theta) \rightarrow E_1(r,\theta) =  0

In [39]:
show(deins[0,3] == kappa*dTdd[0,3])                                                           #Eq 48

-0.5*((0.25*diff(lamb(r), r)*diff(nu(r), r) - 0.25*diff(nu(r), r)^2 + 1.0*diff(lamb(r), r)/r - 0.5*diff(nu(r), r, r))*e^(-lamb(r)) - (-0.25*e^(-lamb(r) + nu(r))*diff(lamb(r), r)*diff(nu(r), r) + 0.25*e^(-lamb(r) + nu(r))*diff(nu(r), r)^2 + 0.5*e^(-lamb(r) + nu(r))*diff(nu(r), r, r) + 1.0*e^(-lamb(r) + nu(r))*diff(nu(r), r)/r)*e^(-nu(r)) + (0.5*r*e^(-lamb(r))*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*diff(nu(r), r) - 1.0*e^(-lamb(r)) + 1.0)/r^2 + (0.5*r*e^(-lamb(r))*sin(th)^2*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*sin(th)^2*diff(nu(r), r) - 1.0*e^(-lamb(r))*sin(th)^2 + 1.0*sin(th)^2)/(r^2*sin(th)^2))*r^2*omega(r, th)*sin(th)^2 - 1.0*r*e^(-lamb(r))*omega(r, th)*sin(th)^2*diff(nu(r), r) == 8*pi*(r^2*P(r, th)*omega(r, th)*sin(th)^2 - (r^2*Omega1(eps)*e^(-1/2*nu(r))*sin(th)^2 + r^2*e^(-1/2*nu(r))*omega(r, th)*sin(th)^2)*(E(r, th) + P(r, th))*e^(1/2*nu(r)))

In [26]:
# Para la segunda derivada tenemos

show(ddeins[0,0]," = ", kappa*ddTdd[0,0])

2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^(-lamb(r) + 2*nu(r))*diff(nu(r), r)^2 - 1.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r)^2 - 2.0*(e^lamb(r)*m(r, th)*diff(lamb(r), r) + e^lamb(r)*diff(m(r, th), r))*e^(-2*lamb(r) + nu(r))*diff(nu(r), r) - 2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r, r) - 1.0*(r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*((0.25*diff(lamb(r), r)*diff(nu(r), r) - 0.25*diff(nu(r), r)^2 + 1.0*diff(lamb(r), r)/r - 0.5*diff(nu(r), r, r))*e^(-lamb(r)) - (-0.25*e^(-lamb(r) + nu(r))*diff(lamb(r), r)*diff(nu(r), r) + 0.25*e^(-lamb(r) + nu(r))*diff(nu(r), r)^2 + 0.5*e^(-lamb(r) + nu(r))*diff(nu(r), r, r) + 1.0*e^(-lamb(r) + nu(r))*diff(nu(r), r)/r)*e^(-nu(r)) + (0.5*r*e^(-lamb(r))*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*diff(nu(r), r) - 1.0*e^(-lamb(r)) + 1.0)/r^2 + (0.5*r*e^(-lamb(r))*sin(th)^2*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*sin(th)^2*diff(nu(r), r) - 1.0*e^(-lamb(r))*sin(th)^2 + 1.0*sin(th)^2)/(r^2*sin(th)^2)) + 0.5*(-4.0*r*e^(-lamb(r) - nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - 4*(0.25*diff(lamb(r), r)*diff(nu(r), r) - 0.25*diff(nu(r), r)^2 + 1.0*diff(lamb(r), r)/r - 0.5*diff(nu(r), r, r))*e^(-lamb(r))*m(r, th) + 2*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*(-0.25*e^(-lamb(r) + nu(r))*diff(lamb(r), r)*diff(nu(r), r) + 0.25*e^(-lamb(r) + nu(r))*diff(nu(r), r)^2 + 0.5*e^(-lamb(r) + nu(r))*diff(nu(r), r, r) + 1.0*e^(-lamb(r) + nu(r))*diff(nu(r), r)/r) + (2.0*r*e^(-nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(lamb(r), r)*diff(nu(r), r) + 2.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r)^2 - 2.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - r^2*e^(-2*nu(r))*omega(r, th)*sin(th)^2*diff(omega(r, th), r) - r*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r))*diff(nu(r), r) + (r^2*omega(r, th)*sin(th)^2*diff(omega(r, th), r) + r*omega(r, th)^2*sin(th)^2 - e^nu(r)*h(r, th)*diff(nu(r), r) - e^nu(r)*diff(h(r, th), r))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r) - 1.0*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 1.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r, r) - 2.0*k(r, th)*diff(lamb(r), r)/r - 4.0*m(r, th)*diff(lamb(r), r)/r - 4.0*k(r, th)/r^2 + 4.0*(r^2*diff(k(r, th), r) + 2*r*k(r, th))/r^3 - 1.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)*diff(lamb(r), r)/(r^3*sin(th)^2) - 2.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4*diff(nu(r), r) - 2*r^4*e^(-nu(r))*omega(r, th)*sin(th)^4*diff(omega(r, th), r) - 4*r^3*e^(-nu(r))*omega(r, th)^2*sin(th)^4 - 2*r^2*sin(th)^2*diff(k(r, th), r) - 4*r*k(r, th)*sin(th)^2)/(r^3*sin(th)^2) - 2.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)/(r^4*sin(th)^2))*e^(-lamb(r)) - (2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^(-lamb(r) + 2*nu(r))*diff(nu(r), r)^2 - 1.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r)^2 - 2.0*(e^lamb(r)*m(r, th)*diff(lamb(r), r) + e^lamb(r)*diff(m(r, th), r))*e^(-2*lamb(r) + nu(r))*diff(nu(r), r) - 2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r, r) - 2.0*e^(-lamb(r) + nu(r))*k(r, th)*diff(nu(r), r)/r - 4.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r)/r - 1.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)*e^(-lamb(r) + nu(r))*diff(nu(r), r)/(r^3*sin(th)^2))*e^(-nu(r)) - 4*(0.5*r*e^(-lamb(r))*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*diff(nu(r), r) - 1.0*e^(-lamb(r)) + 1.0)*k(r, th)

In [27]:
show(ddeins[1,1]," = ", kappa*ddTdd[1,1])

2.0*r*e^(-nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(lamb(r), r)*diff(nu(r), r) + 2.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r)^2 - 2.0*((0.25*diff(lamb(r), r)*diff(nu(r), r) - 0.25*diff(nu(r), r)^2 + 1.0*diff(lamb(r), r)/r - 0.5*diff(nu(r), r, r))*e^(-lamb(r)) - (-0.25*e^(-lamb(r) + nu(r))*diff(lamb(r), r)*diff(nu(r), r) + 0.25*e^(-lamb(r) + nu(r))*diff(nu(r), r)^2 + 0.5*e^(-lamb(r) + nu(r))*diff(nu(r), r, r) + 1.0*e^(-lamb(r) + nu(r))*diff(nu(r), r)/r)*e^(-nu(r)) + (0.5*r*e^(-lamb(r))*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*diff(nu(r), r) - 1.0*e^(-lamb(r)) + 1.0)/r^2 + (0.5*r*e^(-lamb(r))*sin(th)^2*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*sin(th)^2*diff(nu(r), r) - 1.0*e^(-lamb(r))*sin(th)^2 + 1.0*sin(th)^2)/(r^2*sin(th)^2))*e^lamb(r)*m(r, th) - 2.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - r^2*e^(-2*nu(r))*omega(r, th)*sin(th)^2*diff(omega(r, th), r) - r*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r))*diff(nu(r), r) + (r^2*omega(r, th)*sin(th)^2*diff(omega(r, th), r) + r*omega(r, th)^2*sin(th)^2 - e^nu(r)*h(r, th)*diff(nu(r), r) - e^nu(r)*diff(h(r, th), r))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r) - 1.0*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 1.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r, r) - 0.5*(-4.0*r*e^(-lamb(r) - nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - 4*(0.25*diff(lamb(r), r)*diff(nu(r), r) - 0.25*diff(nu(r), r)^2 + 1.0*diff(lamb(r), r)/r - 0.5*diff(nu(r), r, r))*e^(-lamb(r))*m(r, th) + 2*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*(-0.25*e^(-lamb(r) + nu(r))*diff(lamb(r), r)*diff(nu(r), r) + 0.25*e^(-lamb(r) + nu(r))*diff(nu(r), r)^2 + 0.5*e^(-lamb(r) + nu(r))*diff(nu(r), r, r) + 1.0*e^(-lamb(r) + nu(r))*diff(nu(r), r)/r) + (2.0*r*e^(-nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(lamb(r), r)*diff(nu(r), r) + 2.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r)^2 - 2.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - r^2*e^(-2*nu(r))*omega(r, th)*sin(th)^2*diff(omega(r, th), r) - r*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r))*diff(nu(r), r) + (r^2*omega(r, th)*sin(th)^2*diff(omega(r, th), r) + r*omega(r, th)^2*sin(th)^2 - e^nu(r)*h(r, th)*diff(nu(r), r) - e^nu(r)*diff(h(r, th), r))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r) - 1.0*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 1.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r, r) - 2.0*k(r, th)*diff(lamb(r), r)/r - 4.0*m(r, th)*diff(lamb(r), r)/r - 4.0*k(r, th)/r^2 + 4.0*(r^2*diff(k(r, th), r) + 2*r*k(r, th))/r^3 - 1.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)*diff(lamb(r), r)/(r^3*sin(th)^2) - 2.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4*diff(nu(r), r) - 2*r^4*e^(-nu(r))*omega(r, th)*sin(th)^4*diff(omega(r, th), r) - 4*r^3*e^(-nu(r))*omega(r, th)^2*sin(th)^4 - 2*r^2*sin(th)^2*diff(k(r, th), r) - 4*r*k(r, th)*sin(th)^2)/(r^3*sin(th)^2) - 2.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)/(r^4*sin(th)^2))*e^(-lamb(r)) - (2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^(-lamb(r) + 2*nu(r))*diff(nu(r), r)^2 - 1.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r)^2 - 2.0*(e^lamb(r)*m

In [28]:
show(ddeins[2,2]," = ", kappa*ddTdd[2,2])

-2.0*((0.25*diff(lamb(r), r)*diff(nu(r), r) - 0.25*diff(nu(r), r)^2 + 1.0*diff(lamb(r), r)/r - 0.5*diff(nu(r), r, r))*e^(-lamb(r)) - (-0.25*e^(-lamb(r) + nu(r))*diff(lamb(r), r)*diff(nu(r), r) + 0.25*e^(-lamb(r) + nu(r))*diff(nu(r), r)^2 + 0.5*e^(-lamb(r) + nu(r))*diff(nu(r), r, r) + 1.0*e^(-lamb(r) + nu(r))*diff(nu(r), r)/r)*e^(-nu(r)) + (0.5*r*e^(-lamb(r))*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*diff(nu(r), r) - 1.0*e^(-lamb(r)) + 1.0)/r^2 + (0.5*r*e^(-lamb(r))*sin(th)^2*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*sin(th)^2*diff(nu(r), r) - 1.0*e^(-lamb(r))*sin(th)^2 + 1.0*sin(th)^2)/(r^2*sin(th)^2))*r^2*k(r, th) - 4.0*r*e^(-lamb(r))*m(r, th)*diff(lamb(r), r) + 1.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*r*e^(-lamb(r) + nu(r))*diff(nu(r), r) + 2.0*r*e^(-lamb(r))*m(r, th)*diff(nu(r), r) - 0.5*(-4.0*r*e^(-lamb(r) - nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - 4*(0.25*diff(lamb(r), r)*diff(nu(r), r) - 0.25*diff(nu(r), r)^2 + 1.0*diff(lamb(r), r)/r - 0.5*diff(nu(r), r, r))*e^(-lamb(r))*m(r, th) + 2*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*(-0.25*e^(-lamb(r) + nu(r))*diff(lamb(r), r)*diff(nu(r), r) + 0.25*e^(-lamb(r) + nu(r))*diff(nu(r), r)^2 + 0.5*e^(-lamb(r) + nu(r))*diff(nu(r), r, r) + 1.0*e^(-lamb(r) + nu(r))*diff(nu(r), r)/r) + (2.0*r*e^(-nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(lamb(r), r)*diff(nu(r), r) + 2.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r)^2 - 2.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - r^2*e^(-2*nu(r))*omega(r, th)*sin(th)^2*diff(omega(r, th), r) - r*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r))*diff(nu(r), r) + (r^2*omega(r, th)*sin(th)^2*diff(omega(r, th), r) + r*omega(r, th)^2*sin(th)^2 - e^nu(r)*h(r, th)*diff(nu(r), r) - e^nu(r)*diff(h(r, th), r))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r) - 1.0*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 1.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r, r) - 2.0*k(r, th)*diff(lamb(r), r)/r - 4.0*m(r, th)*diff(lamb(r), r)/r - 4.0*k(r, th)/r^2 + 4.0*(r^2*diff(k(r, th), r) + 2*r*k(r, th))/r^3 - 1.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)*diff(lamb(r), r)/(r^3*sin(th)^2) - 2.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4*diff(nu(r), r) - 2*r^4*e^(-nu(r))*omega(r, th)*sin(th)^4*diff(omega(r, th), r) - 4*r^3*e^(-nu(r))*omega(r, th)^2*sin(th)^4 - 2*r^2*sin(th)^2*diff(k(r, th), r) - 4*r*k(r, th)*sin(th)^2)/(r^3*sin(th)^2) - 2.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)/(r^4*sin(th)^2))*e^(-lamb(r)) - (2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^(-lamb(r) + 2*nu(r))*diff(nu(r), r)^2 - 1.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r)^2 - 2.0*(e^lamb(r)*m(r, th)*diff(lamb(r), r) + e^lamb(r)*diff(m(r, th), r))*e^(-2*lamb(r) + nu(r))*diff(nu(r), r) - 2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r, r) - 2.0*e^(-lamb(r) + nu(r))*k(r, th)*diff(nu(r), r)/r - 4.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r)/r - 1.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)*e^(-lamb(r) + nu(r))*diff(nu(r), r)/(r^3*sin(th)^2))*e^(-nu(r)) - 4*(0.5*r*e^(-lamb(r))*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*diff(nu(r), r) - 1.0*e^(-lamb(r)) + 1.0)*k(r, th)/r^2 + (-4.0*r*e^(-lamb(r))*m(r, th)*diff(lamb(r), r) + 1.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*r*e^(-lamb(r) + nu(r))*diff(nu(r), r) + 2.0*r*e^(-lamb(r))*m(r, 

In [29]:
show(ddeins[3,3]," = ", kappa*ddTdd[3,3])

-2.0*((0.25*diff(lamb(r), r)*diff(nu(r), r) - 0.25*diff(nu(r), r)^2 + 1.0*diff(lamb(r), r)/r - 0.5*diff(nu(r), r, r))*e^(-lamb(r)) - (-0.25*e^(-lamb(r) + nu(r))*diff(lamb(r), r)*diff(nu(r), r) + 0.25*e^(-lamb(r) + nu(r))*diff(nu(r), r)^2 + 0.5*e^(-lamb(r) + nu(r))*diff(nu(r), r, r) + 1.0*e^(-lamb(r) + nu(r))*diff(nu(r), r)/r)*e^(-nu(r)) + (0.5*r*e^(-lamb(r))*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*diff(nu(r), r) - 1.0*e^(-lamb(r)) + 1.0)/r^2 + (0.5*r*e^(-lamb(r))*sin(th)^2*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*sin(th)^2*diff(nu(r), r) - 1.0*e^(-lamb(r))*sin(th)^2 + 1.0*sin(th)^2)/(r^2*sin(th)^2))*r^2*k(r, th)*sin(th)^2 - 4.0*r*e^(-lamb(r))*m(r, th)*sin(th)^2*diff(lamb(r), r) + 1.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*r*e^(-lamb(r) + nu(r))*sin(th)^2*diff(nu(r), r) + 2.0*r*e^(-lamb(r))*m(r, th)*sin(th)^2*diff(nu(r), r) - 0.5*(-4.0*r*e^(-lamb(r) - nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - 4*(0.25*diff(lamb(r), r)*diff(nu(r), r) - 0.25*diff(nu(r), r)^2 + 1.0*diff(lamb(r), r)/r - 0.5*diff(nu(r), r, r))*e^(-lamb(r))*m(r, th) + 2*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*(-0.25*e^(-lamb(r) + nu(r))*diff(lamb(r), r)*diff(nu(r), r) + 0.25*e^(-lamb(r) + nu(r))*diff(nu(r), r)^2 + 0.5*e^(-lamb(r) + nu(r))*diff(nu(r), r, r) + 1.0*e^(-lamb(r) + nu(r))*diff(nu(r), r)/r) + (2.0*r*e^(-nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(lamb(r), r)*diff(nu(r), r) + 2.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r)^2 - 2.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - r^2*e^(-2*nu(r))*omega(r, th)*sin(th)^2*diff(omega(r, th), r) - r*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r))*diff(nu(r), r) + (r^2*omega(r, th)*sin(th)^2*diff(omega(r, th), r) + r*omega(r, th)^2*sin(th)^2 - e^nu(r)*h(r, th)*diff(nu(r), r) - e^nu(r)*diff(h(r, th), r))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r) - 1.0*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 1.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r, r) - 2.0*k(r, th)*diff(lamb(r), r)/r - 4.0*m(r, th)*diff(lamb(r), r)/r - 4.0*k(r, th)/r^2 + 4.0*(r^2*diff(k(r, th), r) + 2*r*k(r, th))/r^3 - 1.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)*diff(lamb(r), r)/(r^3*sin(th)^2) - 2.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4*diff(nu(r), r) - 2*r^4*e^(-nu(r))*omega(r, th)*sin(th)^4*diff(omega(r, th), r) - 4*r^3*e^(-nu(r))*omega(r, th)^2*sin(th)^4 - 2*r^2*sin(th)^2*diff(k(r, th), r) - 4*r*k(r, th)*sin(th)^2)/(r^3*sin(th)^2) - 2.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)/(r^4*sin(th)^2))*e^(-lamb(r)) - (2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^(-lamb(r) + 2*nu(r))*diff(nu(r), r)^2 - 1.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r)^2 - 2.0*(e^lamb(r)*m(r, th)*diff(lamb(r), r) + e^lamb(r)*diff(m(r, th), r))*e^(-2*lamb(r) + nu(r))*diff(nu(r), r) - 2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r, r) - 2.0*e^(-lamb(r) + nu(r))*k(r, th)*diff(nu(r), r)/r - 4.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r)/r - 1.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)*e^(-lamb(r) + nu(r))*diff(nu(r), r)/(r^3*sin(th)^2))*e^(-nu(r)) - 4*(0.5*r*e^(-lamb(r))*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*diff(nu(r), r) - 1.0*e^(-lamb(r)) + 1.0)*k(r, th)/r^2 + (-4.0*r*e^(-lamb(r))*m(r, th)*diff(lamb(r), r) + 1.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*r*e^(-lamb(r) + nu(r))*d

In [30]:
show(ddeins[0,3]," = ", kappa*ddTdd[0,3])

0 ' = ' 8*pi*(2*r^2*P1(r)*omega(r, th)*sin(th)^2 - 2*(r^2*Omega1(eps)*e^(-1/2*nu(r))*sin(th)^2 + r^2*e^(-1/2*nu(r))*omega(r, th)*sin(th)^2)*(E(r, th) + P(r, th))*e^nu(r)*u10(r) - (r^2*Omega2(eps)*e^(-1/2*nu(r))*sin(th)^2 + 2*r^2*Omega1(eps)*sin(th)^2*u10(r) + 2*r^2*omega(r, th)*sin(th)^2*u10(r))*(E(r, th) + P(r, th))*e^(1/2*nu(r)) - 2*(r^2*Omega1(eps)*e^(-1/2*nu(r))*sin(th)^2 + r^2*e^(-1/2*nu(r))*omega(r, th)*sin(th)^2)*(E1(r) + P1(r))*e^(1/2*nu(r)))

In [31]:
show(ddeins[1,2]," = ", kappa*ddTdd[1,2])

1.0*r^2*cos(th)*e^(-nu(r))*omega(r, th)^2*sin(th)*diff(nu(r), r) + 2.0*(r^2*cos(th)*e^(-2*nu(r))*omega(r, th)^2*sin(th) + r^2*e^(-2*nu(r))*omega(r, th)*sin(th)^2*diff(omega(r, th), th) - (r^2*cos(th)*omega(r, th)^2*sin(th) + r^2*omega(r, th)*sin(th)^2*diff(omega(r, th), th) - e^nu(r)*diff(h(r, th), th))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r) + 2.0*diff(lamb(r), r)*diff(m(r, th), th) - 4.0*cos(th)*k(r, th)/(r*sin(th)) - 2.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)*cos(th)/(r^3*sin(th)^3) + 4.0*(2*r^4*cos(th)*e^(-nu(r))*omega(r, th)^2*sin(th)^3 + r^4*e^(-nu(r))*omega(r, th)*sin(th)^4*diff(omega(r, th), th) + 2*r^2*cos(th)*k(r, th)*sin(th) + r^2*sin(th)^2*diff(k(r, th), th))/(r^3*sin(th)^2) ' = ' 0

In [32]:
# Aplicamos barotropic EoS
# Utilizando (6.11) de Reina con P1=0 y E1=0
E2*diff(P,r) - P2*diff(E,r) == 0

-P2(r)*diff(E(r, th), r) + E2(r)*diff(P(r, th), r) == 0

In [34]:
ddeins[0,0]

2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^(-lamb(r) + 2*nu(r))*diff(nu(r), r)^2 - 1.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r)^2 - 2.0*(e^lamb(r)*m(r, th)*diff(lamb(r), r) + e^lamb(r)*diff(m(r, th), r))*e^(-2*lamb(r) + nu(r))*diff(nu(r), r) - 2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r, r) - 1.0*(r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*((0.25*diff(lamb(r), r)*diff(nu(r), r) - 0.25*diff(nu(r), r)^2 + 1.0*diff(lamb(r), r)/r - 0.5*diff(nu(r), r, r))*e^(-lamb(r)) - (-0.25*e^(-lamb(r) + nu(r))*diff(lamb(r), r)*diff(nu(r), r) + 0.25*e^(-lamb(r) + nu(r))*diff(nu(r), r)^2 + 0.5*e^(-lamb(r) + nu(r))*diff(nu(r), r, r) + 1.0*e^(-lamb(r) + nu(r))*diff(nu(r), r)/r)*e^(-nu(r)) + (0.5*r*e^(-lamb(r))*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*diff(nu(r), r) - 1.0*e^(-lamb(r)) + 1.0)/r^2 + (0.5*r*e^(-lamb(r))*sin(th)^2*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*sin(th)^2*diff(nu(r), r) - 1.0*e^(-lamb(r))*sin(th)^2 + 1.0*sin(th)^2)/(r^2*sin(th)^2)) + 0.5*(-4.0*r*e^(-lamb(r) - nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - 4*(0.25*diff(lamb(r), r)*diff(nu(r), r) - 0.25*diff(nu(r), r)^2 + 1.0*diff(lamb(r), r)/r - 0.5*diff(nu(r), r, r))*e^(-lamb(r))*m(r, th) + 2*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*(-0.25*e^(-lamb(r) + nu(r))*diff(lamb(r), r)*diff(nu(r), r) + 0.25*e^(-lamb(r) + nu(r))*diff(nu(r), r)^2 + 0.5*e^(-lamb(r) + nu(r))*diff(nu(r), r, r) + 1.0*e^(-lamb(r) + nu(r))*diff(nu(r), r)/r) + (2.0*r*e^(-nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(lamb(r), r)*diff(nu(r), r) + 2.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r)^2 - 2.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - r^2*e^(-2*nu(r))*omega(r, th)*sin(th)^2*diff(omega(r, th), r) - r*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r))*diff(nu(r), r) + (r^2*omega(r, th)*sin(th)^2*diff(omega(r, th), r) + r*omega(r, th)^2*sin(th)^2 - e^nu(r)*h(r, th)*diff(nu(r), r) - e^nu(r)*diff(h(r, th), r))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r) - 1.0*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 1.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r, r) - 2.0*k(r, th)*diff(lamb(r), r)/r - 4.0*m(r, th)*diff(lamb(r), r)/r - 4.0*k(r, th)/r^2 + 4.0*(r^2*diff(k(r, th), r) + 2*r*k(r, th))/r^3 - 1.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)*diff(lamb(r), r)/(r^3*sin(th)^2) - 2.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4*diff(nu(r), r) - 2*r^4*e^(-nu(r))*omega(r, th)*sin(th)^4*diff(omega(r, th), r) - 4*r^3*e^(-nu(r))*omega(r, th)^2*sin(th)^4 - 2*r^2*sin(th)^2*diff(k(r, th), r) - 4*r*k(r, th)*sin(th)^2)/(r^3*sin(th)^2) - 2.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)/(r^4*sin(th)^2))*e^(-lamb(r)) - (2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^(-lamb(r) + 2*nu(r))*diff(nu(r), r)^2 - 1.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r)^2 - 2.0*(e^lamb(r)*m(r, th)*diff(lamb(r), r) + e^lamb(r)*diff(m(r, th), r))*e^(-2*lamb(r) + nu(r))*diff(nu(r), r) - 2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r, r) - 2.0*e^(-lamb(r) + nu(r))*k(r, th)*diff(nu(r), r)/r - 4.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r)/r - 1.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)*e^(-lamb(r) + nu(r))*diff(nu(r), r)/(r^3*sin(th)^2))*e^(-nu(r)) - 4*(0.5*r*e^(-lamb(r))*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*diff(nu(r), r) - 1.0*e^(-lamb(r)) + 1.0)*k(r, th)

In [35]:
h0 = function('h0')(r,th)
k0 = function('k0')(r,th)
m0 = function('m0')(r,th)

h2 = function('h2')(r,th)
k2 = function('k2')(r,th)
m2 = function('m2')(r,th)

m = m0 + m2*(3*cos(th)^2-1)/2
k = k0 + k2*(3*cos(th)^2-1)/2
h = h0 + h2*(3*cos(th)^2-1)/2

In [36]:
ddeins.apply_map(lambda cmp: cmp.subs(h==h0+h2*(3*cos(th)^2-1)/2))
ddeins.apply_map(lambda cmp: cmp.subs(k==k0+k2*(3*cos(th)^2-1)/2))
ddeins.apply_map(lambda cmp: cmp.subs(m==m0+m2*(3*cos(th)^2-1)/2))

[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^(-lamb(r) + 2*nu(r))*diff(nu(r), r)^2 - 1.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r)^2 - 2.0*(e^lamb(r)*m(r, th)*diff(lamb(r), r) + e^lamb(r)*diff(m(r, th), r))*e^(-2*lamb(r) + nu(r))*diff(nu(r), r) - 2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r, r) - 1.0*(r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*((0.25*diff(lamb(r), r)*diff(nu(r), r) - 0.25*diff(nu(r), r)^2 + 1.0*diff(lamb(r), r)/r - 0.5*diff(nu(r), r, r))*e^(-lamb(r)) - (-0.25*e^(-lamb(r) + nu(r))*diff(lamb(r), r)*diff(nu(r), r) + 0.25*e^(-lamb(r) + nu(r))*diff(nu(r), r)^2 + 0.5*e^(-lamb(r) + nu(r))*diff(nu(r), r, r) + 1.0*e^(-lamb(r) + nu(r))*diff(nu(r), r)/r)*e^(-nu(r)) + (0.5*r*e^(-lamb(r))*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*diff(nu(r), r) - 1.0*e^(-lamb(r)) + 1.0)/r^2 + (0.5*r*e^(-lamb(r))*sin(th)^2*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*sin(th)^2*diff(nu(r), r) - 1.0*e^(-lamb(r))*sin(th)^2 + 1.0*sin(th)^2)/(r^2*sin(th)^2)) + 0.5*(-4.0*r*e^(-lamb(r) - nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - 4*(0.25*diff(lamb(r), r)*diff(nu(r), r) - 0.25*diff(nu(r), r)^2 + 1.0*diff(lamb(r), r)/r - 0.5*diff(nu(r), r, r))*e^(-lamb(r))*m(r, th) + 2*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*(-0.25*e^(-lamb(r) + nu(r))*diff(lamb(r), r)*diff(nu(r), r) + 0.25*e^(-lamb(r) + nu(r))*diff(nu(r), r)^2 + 0.5*e^(-lamb(r) + nu(r))*diff(nu(r), r, r) + 1.0*e^(-lamb(r) + nu(r))*diff(nu(r), r)/r) + (2.0*r*e^(-nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(lamb(r), r)*diff(nu(r), r) + 2.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r)^2 - 2.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - r^2*e^(-2*nu(r))*omega(r, th)*sin(th)^2*diff(omega(r, th), r) - r*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r))*diff(nu(r), r) + (r^2*omega(r, th)*sin(th)^2*diff(omega(r, th), r) + r*omega(r, th)^2*sin(th)^2 - e^nu(r)*h(r, th)*diff(nu(r), r) - e^nu(r)*diff(h(r, th), r))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r) - 1.0*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 1.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r, r) - 2.0*k(r, th)*diff(lamb(r), r)/r - 4.0*m(r, th)*diff(lamb(r), r)/r - 4.0*k(r, th)/r^2 + 4.0*(r^2*diff(k(r, th), r) + 2*r*k(r, th))/r^3 - 1.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)*diff(lamb(r), r)/(r^3*sin(th)^2) - 2.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4*diff(nu(r), r) - 2*r^4*e^(-nu(r))*omega(r, th)*sin(th)^4*diff(omega(r, th), r) - 4*r^3*e^(-nu(r))*omega(r, th)^2*sin(th)^4 - 2*r^2*sin(th)^2*diff(k(r, th), r) - 4*r*k(r, th)*sin(th)^2)/(r^3*sin(th)^2) - 2.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 

In [37]:
ddeins[0,0].subs(m==m0 + m2*(3*cos(th)^2-1)/2)

2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^(-lamb(r) + 2*nu(r))*diff(nu(r), r)^2 - 1.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r)^2 - 2.0*(e^lamb(r)*m(r, th)*diff(lamb(r), r) + e^lamb(r)*diff(m(r, th), r))*e^(-2*lamb(r) + nu(r))*diff(nu(r), r) - 2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r, r) - 1.0*(r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*((0.25*diff(lamb(r), r)*diff(nu(r), r) - 0.25*diff(nu(r), r)^2 + 1.0*diff(lamb(r), r)/r - 0.5*diff(nu(r), r, r))*e^(-lamb(r)) - (-0.25*e^(-lamb(r) + nu(r))*diff(lamb(r), r)*diff(nu(r), r) + 0.25*e^(-lamb(r) + nu(r))*diff(nu(r), r)^2 + 0.5*e^(-lamb(r) + nu(r))*diff(nu(r), r, r) + 1.0*e^(-lamb(r) + nu(r))*diff(nu(r), r)/r)*e^(-nu(r)) + (0.5*r*e^(-lamb(r))*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*diff(nu(r), r) - 1.0*e^(-lamb(r)) + 1.0)/r^2 + (0.5*r*e^(-lamb(r))*sin(th)^2*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*sin(th)^2*diff(nu(r), r) - 1.0*e^(-lamb(r))*sin(th)^2 + 1.0*sin(th)^2)/(r^2*sin(th)^2)) + 0.5*(-4.0*r*e^(-lamb(r) - nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - 4*(0.25*diff(lamb(r), r)*diff(nu(r), r) - 0.25*diff(nu(r), r)^2 + 1.0*diff(lamb(r), r)/r - 0.5*diff(nu(r), r, r))*e^(-lamb(r))*m(r, th) + 2*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*(-0.25*e^(-lamb(r) + nu(r))*diff(lamb(r), r)*diff(nu(r), r) + 0.25*e^(-lamb(r) + nu(r))*diff(nu(r), r)^2 + 0.5*e^(-lamb(r) + nu(r))*diff(nu(r), r, r) + 1.0*e^(-lamb(r) + nu(r))*diff(nu(r), r)/r) + (2.0*r*e^(-nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(lamb(r), r)*diff(nu(r), r) + 2.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r)^2 - 2.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2*diff(nu(r), r) - r^2*e^(-2*nu(r))*omega(r, th)*sin(th)^2*diff(omega(r, th), r) - r*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r))*diff(nu(r), r) + (r^2*omega(r, th)*sin(th)^2*diff(omega(r, th), r) + r*omega(r, th)^2*sin(th)^2 - e^nu(r)*h(r, th)*diff(nu(r), r) - e^nu(r)*diff(h(r, th), r))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r) - 1.0*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 1.0*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^nu(r)*diff(nu(r), r, r) - 2.0*k(r, th)*diff(lamb(r), r)/r - 4.0*m(r, th)*diff(lamb(r), r)/r - 4.0*k(r, th)/r^2 + 4.0*(r^2*diff(k(r, th), r) + 2*r*k(r, th))/r^3 - 1.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)*diff(lamb(r), r)/(r^3*sin(th)^2) - 2.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4*diff(nu(r), r) - 2*r^4*e^(-nu(r))*omega(r, th)*sin(th)^4*diff(omega(r, th), r) - 4*r^3*e^(-nu(r))*omega(r, th)^2*sin(th)^4 - 2*r^2*sin(th)^2*diff(k(r, th), r) - 4*r*k(r, th)*sin(th)^2)/(r^3*sin(th)^2) - 2.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)/(r^4*sin(th)^2))*e^(-lamb(r)) - (2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) + 0.5*(r^2*e^(-2*nu(r))*omega(r, th)^2*sin(th)^2 - (r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-2*nu(r)))*e^(-lamb(r) + 2*nu(r))*diff(nu(r), r)^2 - 1.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r)^2 - 2.0*(e^lamb(r)*m(r, th)*diff(lamb(r), r) + e^lamb(r)*diff(m(r, th), r))*e^(-2*lamb(r) + nu(r))*diff(nu(r), r) - 2.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r, r) - 2.0*e^(-lamb(r) + nu(r))*k(r, th)*diff(nu(r), r)/r - 4.0*e^(-lamb(r) + nu(r))*m(r, th)*diff(nu(r), r)/r - 1.0*(r^4*e^(-nu(r))*omega(r, th)^2*sin(th)^4 + 2*r^2*k(r, th)*sin(th)^2)*e^(-lamb(r) + nu(r))*diff(nu(r), r)/(r^3*sin(th)^2))*e^(-nu(r)) - 4*(0.5*r*e^(-lamb(r))*diff(lamb(r), r) - 0.5*r*e^(-lamb(r))*diff(nu(r), r) - 1.0*e^(-lamb(r)) + 1.0)*k(r, th)